In [15]:
# Create travel paths for mainstem runs

In [16]:
import geopandas as gpd
from shapely.ops import linemerge
from shapely.geometry import MultiLineString, LineString
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [26]:
# Specify the path where you want to save the GeoPackage file
str_hydrofabric_path = r'E:\sample_2d_output\model_hydrofabric.gpkg'

# Read the nextgen hydrofabric
gdf_flowpaths = gpd.read_file(str_hydrofabric_path, layer='01_stream_lines')

# Read the nextgen hydrofabric
gdf_flowpoints = gpd.read_file(str_hydrofabric_path, layer='02_flow_points')

In [18]:
# Dissolve the travel paths (flowpaths) by 'mainstem' attribute
gdf_mainstems = gdf_flowpaths.dissolve(by='mainstem')

# Reset the index
gdf_mainstems.reset_index(inplace=True)

# Keep only the 'mainstem' and 'geometry' columns
gdf_mainstems = gdf_mainstems[['mainstem', 'geometry']]

# Assuming gdf_mainstems is a GeoDataFrame with geometry column containing LineString or MultiLineString
for idx, row in gdf_mainstems.iterrows():
    geom = row['geometry']
    if isinstance(geom, MultiLineString):
        merged_line = linemerge(geom)
        if merged_line.is_empty:
            pass
        else:
            # Replace the MultiLineString with the merged LineString
            gdf_mainstems.at[idx, 'geometry'] = merged_line

In [19]:
# Clip mainstems by the rivers that are inside "NHD waterbodies"

In [20]:
# Filter rows where rl_NHDWaterbodyComID is not null
gdf_waterbody_flowpaths = gdf_flowpaths[gdf_flowpaths['rl_NHDWaterbodyComID'].notnull()]

# Dissolve the travel paths by 'mainstem' attribute
gdf_waterbody_flowpaths_disolve = gdf_waterbody_flowpaths.dissolve(by='mainstem')

# Reset the index
gdf_waterbody_flowpaths_disolve.reset_index(inplace=True)

# Keep only the 'mainstem' and 'geometry' columns
gdf_waterbody_flowpaths_disolve = gdf_waterbody_flowpaths_disolve[['mainstem', 'geometry']]

# Assuming gdf_mainstems is a GeoDataFrame with geometry column containing LineString or MultiLineString
for idx, row in gdf_waterbody_flowpaths_disolve.iterrows():
    geom = row['geometry']
    if isinstance(geom, MultiLineString):
        merged_line = linemerge(geom)
        if merged_line.is_empty:
            pass
        else:
            # Replace the MultiLineString with the merged LineString
            gdf_waterbody_flowpaths_disolve.at[idx, 'geometry'] = merged_line

In [21]:
# Clip mainstems by the rivers that are inside "NHD waterbodies"

In [22]:
gdf_mainstems_revised = gdf_mainstems

# --------
def fn_convert_to_list_of_linestrings(geometry):
    if isinstance(geometry, MultiLineString):
        result = []
        for part in geometry.geoms:
            result.append(LineString(part.coords))
        return result
    elif isinstance(geometry, LineString):
        return [geometry]
    else:
        return []
# --------

for index, row in gdf_waterbody_flowpaths_disolve.iterrows():
    
    # Extract the mainstem value
    mainstem_value = row['mainstem']
    
    # Extract the corresponding linestring from gdf_mainstems
    mainstem_linestring = gdf_mainstems[gdf_mainstems['mainstem'] == mainstem_value].geometry.iloc[0]
    
    # Clip the linestring from gdf_waterbody_flowpaths with the linestring from gdf_waterbody_flowpaths
    mainstem_differance = mainstem_linestring.difference(row.geometry)
    
    list_linestrings = fn_convert_to_list_of_linestrings(mainstem_differance)
    
    if len(list_linestrings) > 0:
        # Delete row in gdf_mainstems_revied where 'mainstem' = mainstem_value
        gdf_mainstems_revised = gdf_mainstems_revised[gdf_mainstems_revised['mainstem'] != mainstem_value]
        
        # Append items in list_linestrings to gdf_mainstems_revied as new rows
        for linestring in list_linestrings:
            #gdf_mainstems_revied = gdf_mainstems_revised.append({'mainstem': mainstem_value, 'geometry': linestring},
            #                                                   ignore_index=True)
            gdf_mainstems_revised = pd.concat([gdf_mainstems_revised,
                                               pd.DataFrame({'mainstem': [mainstem_value], 'geometry': [linestring]})],
                                              ignore_index=True)

gdf_mainstems_revised.crs = gdf_mainstems.crs

C:\Users\civil\anaconda3\envs\tx-bridge\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Users\civil\anaconda3\envs\tx-bridge\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [23]:
# for each item in gdf_mainstems_revised, get the point nearest to the startpoint

In [24]:
# create a geodataframe of points for the beginning of each item in gdf_mainstems_revised (that contains linestrings)

# Create a list to store the starting points
starting_points = []

# Iterate over each row in the GeoDataFrame
for index, row in gdf_mainstems_revised.iterrows():
    # Extract the starting point of the LineString
    start_point = Point(row['geometry'].coords[0])
    starting_points.append(start_point)

# Create a new GeoDataFrame from the starting points
gdf_starting_points = gpd.GeoDataFrame(geometry=starting_points, crs=gdf_mainstems_revised.crs)

gdf_starting_points['mainstem'] = gdf_mainstems_revised['mainstem']

In [25]:
gdf_starting_points.explore()